# Setup

In [ ]:
%pip install OpenAI markdown mdx_truly_sane_lists weasyprint python-dotenv pango pdflatex ipywidgets

In [ ]:
import os

from dotenv import load_dotenv
load_dotenv()  # Load the environment variables from .env file

Set up the OpenAI Assistant

In [ ]:
from openai import OpenAI
client = OpenAI()

#  get the prompt from the file system
with open('prompt.md', 'r') as file:
  # Read the entire content of the file into a single string variable
  prompt = file.read()


# update the assistant with the latest prompt
my_assistant = client.beta.assistants.update(
  "asst_3WQiw8VJ91XWikievImBqwrZ",
  instructions=prompt,
  name="Interview Playbook Generator",
  tools=[{"type": "retrieval"}],
  model="gpt-4-1106-preview"
)

In [ ]:
# create a thread
my_thread = client.beta.threads.create()

import time

def create_run(client, my_thread, my_assistant, content, file_ids=None):
    """
    Create a message, run it, and display the response.

    Args:
    client (Client): The API client.
    my_thread (Thread): The thread to post the message to.
    my_assistant (Assistant): The assistant to use for the run.
    content (str): The content of the message.
    file_ids (list, optional): List of file IDs to attach to the message.

    Returns:
    str: The response content.

    Raises:
    ValueError: If any of the parameters are invalid.
    RuntimeError: If the run does not complete successfully.
    """

    if not all([client, my_thread, my_assistant, content]):
        raise ValueError("Invalid parameters provided.")

    #extra_instructions = """"""
    extra_instructions = """ ONLY return the content as John Gruber's Markdown format, with no commentary whatsoever.  ALWAYS add a blank line between text and a list. Do not delimit the markdown with backticks or any other delimiter."""


    # Create the message
    thread_message = client.beta.threads.messages.create(
        my_thread.id,
        role="user",
        content=content + extra_instructions,
        file_ids=file_ids or []
    )

    # Hard code the values for max_retries and wait_seconds
    max_retries = 120
    wait_seconds = 0.5

    # Create a run
    run = client.beta.threads.runs.create(
        thread_id=my_thread.id,
        assistant_id=my_assistant.id
    )

    # Check the run's status
    for _ in range(max_retries):
        run = client.beta.threads.runs.retrieve(
            thread_id=my_thread.id,
            run_id=run.id)
        if run.status == "completed":
            break
        time.sleep(wait_seconds)
    else:
        raise RuntimeError("Run did not complete successfully.")

    # Retrieve and return the response
    all_messages = client.beta.threads.messages.list(thread_id=my_thread.id)
    return all_messages.data[0].content[0].text.value + "\n\n"


## Choose example and set up file paths

In [ ]:
import time
import ipywidgets as widgets
# Which example to run
choice_widget = widgets.Dropdown(
  options=['medical-assistant', 'infra-engineer', 'customer-success-manager', 'real-estate-litigator'],
  value='medical-assistant',
  description='Choose an example:'
)
display(choice_widget)

# Wait for 5 seconds for the user to select an option
time.sleep(5)

# Get the selected choice
choice = choice_widget.value

# source files
job_description_file = "examples/" + choice + "/job-description.md"
company_file = "examples/" + choice + "/about-company.md"

# destination files
destination_directory = "examples/" + choice + "/"
html_destination = destination_directory + 'playbook.html'
pdf_destination = destination_directory + 'playbook.pdf'
markdown_destination = destination_directory + 'playbook.md'
word_destination = destination_directory + 'playbook.docx'


# Company Analysis

In [ ]:
# Upload the company description
file_company_values = client.files.create(
  file=open(company_file, "rb"),
  purpose="assistants"
)

# Refresh the context to ensure it has read the files
primer_values = create_run(client,my_thread,my_assistant,"""
Write a summary of the mission, vision and values of the company.
            """, [file_company_values.id])
print(primer_values)

# Job Analysis

In [ ]:

# Open the file and read the contents
with open(job_description_file, 'r') as file:
    # Read the entire content of the file into a single string variable
    job_description = file.read()

# Upload the job description file
file_job_description = client.files.create(
  file=open(job_description_file, "rb"),
  purpose="assistants"
)

# Refresh the context to ensure it has read the files and create context. Not included in final doc.
primer_job = create_run(client,my_thread,my_assistant,"""
Write the job title contained in the job description file.
Then, create the list of competencies, which MUST cover the areas mentioned in the
requirements in the job description but should also incorporate the company values wherever possible.
Where possible, describe a competency in a way that aligns with the company values.
Each competency should have a title followed by a short description.
For each competency, list 3 or 4 subcompetencies and their descriptions to enable more deep exploration.
""", [file_job_description.id])
print(primer_job)

# Create Introduction

In [ ]:
playbook_title = create_run(client,my_thread,my_assistant,"""
Generate the title for the playbook as a simple string with no quotation marks
""")
print(playbook_title)

In [ ]:
introduction = create_run(client,my_thread,my_assistant,"""
Generate the introduction for the guide with no headings
""")
print(introduction)

In [ ]:
interview_structure = create_run(client,my_thread,my_assistant,"""
Generate the Interview Structure with no headings. Include mention of the competencies in the interview structure.
""")
print(interview_structure)

# Create Evaluation Criteria

In [ ]:
eval_criteria = create_run(client,my_thread,my_assistant,"""
Generate the Evaluation Critera section with no headings. Remember to include a blank line before a list.
""")
print(eval_criteria)

# Competencies

In [ ]:
technical_assessment_1 = create_run(client,my_thread,my_assistant,"Generate the first Competency Assessment")
print(technical_assessment_1)

In [ ]:
technical_assessment_2 = create_run(client,my_thread,my_assistant,"""
    Generate the 2nd Technical Assessment. Don't include the main section header.
    """)
print(technical_assessment_2)

In [ ]:
technical_assessment_3 = create_run(client,my_thread,my_assistant,"Generate the 3rd Technical Assessment. Don't include the main section header.")
print(technical_assessment_3)

In [ ]:
technical_assessment_4 = create_run(client,my_thread,my_assistant,"Generate the 4th Technical Assessment. Don't include the main section header.")
print(technical_assessment_4)

In [ ]:
technical_assessment_5 = create_run(client,my_thread,my_assistant,"Generate the 5th Technical Assessment. Don't include the main section header.")
print(technical_assessment_5)

In [ ]:
technical_assessment = technical_assessment_1 + technical_assessment_2 + technical_assessment_3 + technical_assessment_4 + technical_assessment_5

# Create Behavioral Assessment

In [ ]:
behavioral_assessment_1 = create_run(client,my_thread,my_assistant,"Generate the 1st Behavioral Assessment section.")
print(behavioral_assessment_1)

In [ ]:
behavioral_assessment_2 = create_run(client,my_thread,my_assistant,"Generate the 2nd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_2)

In [ ]:
behavioral_assessment_3 = create_run(client,my_thread,my_assistant,"Generate the 3rd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_3)

In [ ]:
behavioral_assessment_4 = create_run(client,my_thread,my_assistant,"Generate the 4th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_4)

In [ ]:
behavioral_assessment_5 = create_run(client,my_thread,my_assistant,"Generate the 5th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_5)

In [ ]:
behavioral_assessment = behavioral_assessment_1 + behavioral_assessment_2 + behavioral_assessment_3 + behavioral_assessment_4 + behavioral_assessment_5

# Create Practical Assignment

In [ ]:
practical_assignment = create_run(client,my_thread,my_assistant,"Generate the Practical Assignment section.")
print(practical_assignment)


# Create Team Interaction

In [ ]:
team_interaction = create_run(client,my_thread,my_assistant,"Generate the Team Interaction section.")
print(team_interaction)

# Create Candidate Q&A

In [ ]:
candidate_questions = create_run(client,my_thread,my_assistant,"""
Generate the Candidate Q&A section. Include example answers.
""")
print(candidate_questions)

# Finalize the Playbook HTML

## Convert the markdown to HTML

We are doing this in chunks so that we can do more useful things with it in the HTML output

In [ ]:
import markdown

def convert_to_html(text):
    return markdown.markdown(text, extensions=['extra','mdx_truly_sane_lists'])

html_job_description = convert_to_html(job_description)
html_playbook_title = convert_to_html(playbook_title)
html_intro = convert_to_html(introduction)
html_interview_structure = convert_to_html(interview_structure)
html_technical_assessment = convert_to_html(technical_assessment)
html_behavioral_assessment = convert_to_html(behavioral_assessment)
html_practical_assignment = convert_to_html(practical_assignment)
html_candidate_questions = convert_to_html(candidate_questions)
html_team_interaction = convert_to_html(team_interaction)
html_eval_criteria = convert_to_html(eval_criteria)

html_technical_assessment_1 = convert_to_html(technical_assessment_1)
html_technical_assessment_2 = convert_to_html(technical_assessment_2)
html_technical_assessment_3 = convert_to_html(technical_assessment_3)
html_technical_assessment_4 = convert_to_html(technical_assessment_4)
html_technical_assessment_5 = convert_to_html(technical_assessment_5)

html_behavioral_assessment_1 = convert_to_html(behavioral_assessment_1)
html_behavioral_assessment_2 = convert_to_html(behavioral_assessment_2)
html_behavioral_assessment_3 = convert_to_html(behavioral_assessment_3)
html_behavioral_assessment_4 = convert_to_html(behavioral_assessment_4)
html_behavioral_assessment_5 = convert_to_html(behavioral_assessment_5)



## Build the HTML

Colors from this [color palette](https://color.adobe.com/PALETTE-mini-06:-Transparent-color-theme-15d8c4a6-b06a-4530-bf5b-3fc8102ccd69)

In [ ]:
# Open the file and read the contents
with open('templates/styles.css', 'r') as file:
    # Read the entire content of the file into a single string variable
    styles = file.read()


html_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div id="rating-line" class="short-dotted-line"></div>
    </div>
</div>"""

html_short_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div class="short-dotted-line"></div>
    </div>
</div>"""

from datetime import datetime

# Getting today's date in a nicely formatted string for US English
publication_date = datetime.now().strftime("%B %d, %Y")


# Open the HTML file and read its content
with open('templates/template.html', 'r') as file:
    html_template = file.read()

# Replace the placeholders with the actual variables
html_content = html_template.format(
    styles=styles,
    html_playbook_title=html_playbook_title,
    publication_date=publication_date,
    html_intro=html_intro,
    html_interview_structure=html_interview_structure,
    html_job_description=html_job_description,
    html_eval_criteria=html_eval_criteria,
    html_technical_assessment_1=html_technical_assessment_1,
    html_short_notes=html_short_notes,
    html_technical_assessment_2=html_technical_assessment_2,
    html_notes=html_notes,
    html_technical_assessment_3=html_technical_assessment_3,
    html_technical_assessment_4=html_technical_assessment_4,
    html_technical_assessment_5=html_technical_assessment_5,
    html_behavioral_assessment_1=html_behavioral_assessment_1,
    html_behavioral_assessment_2=html_behavioral_assessment_2,
    html_behavioral_assessment_3=html_behavioral_assessment_3,
    html_behavioral_assessment_4=html_behavioral_assessment_4,
    html_behavioral_assessment_5=html_behavioral_assessment_5,
    html_practical_assignment=html_practical_assignment,
    html_team_interaction=html_team_interaction,
    html_candidate_questions=html_candidate_questions
)

# Write HTML string to a file
with open(html_destination, 'w') as file:
    file.write(html_content)


# Display the Markdown-formatted text in the notebook
# from IPython.display import display, HTML
# display(HTML(html_content))


# Create the Documents

### Create PDF from HTML

In [ ]:
from weasyprint import HTML

# Convert HTML to PDF using WeasyPrint
#HTML('playbook.html').write_pdf('playbook.pdf', stylesheets=['style.css'], presentational_hints=True)
HTML(html_destination).write_pdf(pdf_destination, stylesheets=['templates/styles.css'], full_fonts=True, hinting=True, presentational_hints=True)



## Create a markdown version

For use in GitHub as an example

In [ ]:
notes_snippet_md = """##### Notes\n
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________\n
Rating (1-5): \\__________\n\n\n"""

from datetime import datetime

# Get the current date and time
now = datetime.now()

# Format the date and time in a nice readable format
formatted_now = now.strftime("%B %d, %Y %-I:%M %p")

print(formatted_now)

playbook_body_md = f"""
## Interview Structure
{interview_structure}

## Job Description
{job_description}

{eval_criteria}

{technical_assessment_1}
{notes_snippet_md}
            
{technical_assessment_2}
{notes_snippet_md}

{technical_assessment_3}
{notes_snippet_md}

{technical_assessment_4}
{notes_snippet_md}
            
{technical_assessment_5}
{notes_snippet_md}
            
{behavioral_assessment_1}
{notes_snippet_md}

{behavioral_assessment_2}
{notes_snippet_md}
            
{behavioral_assessment_3}
{notes_snippet_md}
            
{behavioral_assessment_4}
{notes_snippet_md}
            
{behavioral_assessment_5}
{notes_snippet_md}
            
{practical_assignment}
{notes_snippet_md}

{team_interaction}
{notes_snippet_md}

{candidate_questions}
"""

print(len(playbook_body_md))

# exec_summary = create_run(client,my_thread,my_assistant,f"""
#    Write a summary for this document:
#
#    {playbook_body_md}
#    """)

exec_summary = create_run(client,my_thread,my_assistant,f"""
    Write a detailed introduction for the document. Be clear and concise and don't use flowery language.
    """)

    
print(exec_summary)

In [ ]:
playbook_md = f"""--- 
title: {playbook_title}
author: Created by [Deliverables.ai](https://github.com/androb/deliverables)
date: {formatted_now}
--- 

## Introduction
{exec_summary}
            
{playbook_body_md}
"""


# Write playbook to a text file
with open(markdown_destination, 'w') as file:
    file.write(playbook_md)


### Create Word Document from Markdown

In [2]:
# Convert from markdown to docx
!pandoc -s {markdown_destination} -o {word_destination} --reference-doc="templates/custom-reference.docx" --toc=True --toc-depth=2 --shift-heading-level-by -1

[WARNING] Could not deduce format from file extension 
  Defaulting to markdown
[WARNING] Could not deduce format from file extension 
  Defaulting to html
pandoc: {markdown_destination}: withBinaryFile: does not exist (No such file or directory)


In [1]:
# use this extension to shift headings to differ between HTML and docx
# --shift-heading-level-by -1

# Convert from markdown to PDF via HTML
!pandoc -s {markdown_destination} -o {html_destination} --template='templates/html-reference.html --toc=True
!pandoc --pdf-engine "weasyprint" -s {html_destination} -o {pdf_destination}


/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 1: syntax error: unexpected end of file
[WARNING] Could not deduce format from file extension 
  Defaulting to markdown
[WARNING] Could not deduce format from file extension 
  Defaulting to html
pandoc: {html_destination}: withBinaryFile: does not exist (No such file or directory)


In [ ]:
pptx_destination = destination_directory + "playbook.pptx"
!pandoc -s {markdown_destination} -o {pptx_destination}